In [22]:
import numpy as np

import sys
sys.path.append('.')

from sfepy.base.base import output, IndexedStruct
from sfepy.discrete import (FieldVariable, Material, Integral, Integrals,
                            Equation, Equations, Problem)
from sfepy.discrete.fem import Mesh, FEDomain, Field
from sfepy.terms import Term
from sfepy.discrete.conditions import Conditions, EssentialBC
from sfepy.solvers.ls import ScipyDirect
from sfepy.solvers.nls import Newton
from sfepy.mechanics.matcoefs import stiffness_from_youngpoisson
from sfepy import data_dir
from sfepy.solvers.auto_fallback import AutoDirect



In [27]:
# Build the problem definition.
#mesh = Mesh.from_file(data_dir + '/meshes/2d/its2D.mesh')

mesh = Mesh.from_file('triangle2d.mesh')
domain = FEDomain('domain', mesh)

omega = domain.create_region('Omega', 'all')
top = domain.create_region('Top', 'vertex 3', 'vertex')
bottom_left = domain.create_region('Bottom_Left', 'vertex 1', 'vertex')                                        
bottom_right = domain.create_region('Bottom_Right', 'vertex 2', 'vertex')
left = domain.create_region('Left',
                            'vertices in x < 0.001', 'facet')
bottom = domain.create_region('Bottom',
                                'vertices in y < 0.001', 'facet')
top = domain.create_region('Top', 'vertex 3', 'vertex')

field = Field.from_args('fu', np.float64, 'vector', omega,
                        approx_order=2)

u = FieldVariable('u', 'unknown', field)
v = FieldVariable('v', 'test', field, primary_var_name='u')

# define elastic properties:
# Aluminum properties in SI units
E = 7.0e10  # Young's Modulus in Pa (70 GPa for Aluminum)
nu = 0.33  # Poisson's ratio for Aluminum
D = stiffness_from_youngpoisson(2, young=E, poisson=nu)

aluminum = Material('Aluminum', D=D)

#set up loading:
# Define forces in Newtons
fy = 6 * 1e3  # (kn)
force_ratio = np.tan(np.radians(10))  # Work with an angle of 10 degrees.
fx = fy * force_ratio  # Horizontal force
load = Material('Load', values={'.val' : [fx, fy]})

integral = Integral('i', order=2)
integral0 = Integral('i', order=0)

t1 = Term.new('dw_lin_elastic(Aluminum.D, v, u)',
                integral, omega, Aluminum=aluminum, v=v, u=u)
t2 = Term.new('dw_point_load(Load.val, v)',
                integral0, top, Load=load, v=v)
eq = Equation('balance', t1 - t2)
eqs = Equations([eq])

xsym = EssentialBC('XSym', bottom, {'u.1' : 0.0})
ysym = EssentialBC('YSym', left, {'u.0' : 0.0})

ls = ScipyDirect({})

nls_status = IndexedStruct()
nls = Newton({}, lin_solver=ls, status=nls_status)

pb = Problem('elasticity', equations=eqs)

pb.set_bcs(ebcs=Conditions([xsym, ysym]))

pb.set_solver(nls)

# Solve the problem.
variables = pb.solve()
output(nls_status)

# Postprocess the solution.
out = variables.create_output()

ev = pb.evaluate
strain = ev('ev_cauchy_strain.2.Omega(u)', mode='el_avg')
#out = stress_strain(out, pb, variables, extend=True)
#pb.save_state('its2D_interactive.vtk', out=out)



sfepy: reading mesh (triangle2d.mesh)...
sfepy:   number of vertices: 279
sfepy:   number of cells:
sfepy:     2_3: 487
sfepy: ...done in 0.01 s
sfepy: updating variables...
sfepy: ...done
sfepy: setting up dof connectivities...
sfepy: ...done in 0.00 s
sfepy: matrix shape: (2006, 2006)
sfepy: assembling matrix graph...
sfepy: ...done in 0.00 s
sfepy: matrix structural nonzeros: 43340 (1.08e+00% fill)
sfepy: updating variables...
sfepy: ...done
sfepy: updating materials...
sfepy:     Load
sfepy:     Aluminum
sfepy: ...done in 0.00 s
sfepy: nls: iter: 0, residual: 1.057962e+03 (rel: 1.000000e+00)
sfepy:   residual:    0.00 [s]
sfepy:     matrix:    0.00 [s]
sfepy:      solve:    0.01 [s]
sfepy: nls: iter: 1, residual: 2.804157e-12 (rel: 2.650528e-15)


Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.

sfepy: solved in 1 steps in 0.02 seconds
sfepy: IndexedStruct
  condition:
    0
  err:
    2.804157190841315e-12
  err0:
    1057.9618842507898
  ls_n_iter:
    -1
  n_iter:
    1
  time:
    0.0139307499921415
  time_stats:
    dict with keys: ['residual', 'matrix', 'solve']
sfepy: equation "tmp":
sfepy: ev_cauchy_strain.2.Omega(u)
sfepy: updating materials...
sfepy: ...done in 0.00 s


In [18]:
import sfepy
print(sfepy.__version__)


2024.2
